# Adding a new element

Our goal is to add a new element to the incompressible solver! For this we have to add the following class to the file *dream/incompressible.py*

```python

class HDivHDG(IncompressibleFiniteElement):
    name: str = "HDivHDG"

    def add_finite_element_spaces(self, spaces: dict[str, ngs.FESpace]):
        bnds = self.root.bcs.get_region(Wall, Inflow, as_pattern=True)

        U = ...
        Uhat = ...
        P = ...

        if self.root.bcs.has_condition(Periodic):
            U = ngs.Periodic(U)
            Uhat = ngs.Periodic(Uhat)

        spaces['u'] = U
        spaces['p'] = P
        spaces['uhat'] = Uhat

    def add_symbolic_stokes_form(self, blf: Integrals, lf: Integrals):
        u, v = self.TnT['u']
        uhat, vhat = self.TnT['uhat']
        p, q = self.TnT['p']

        U = self.get_incompressible_state([u, p])
        V = self.get_incompressible_state([v, q])

        n = self.mesh.normal
        h = self.mesh.meshsize
        alpha = 10

        Re = self.root.reynolds_number
        nu = self.root.kinematic_viscosity(U)
        mu = 2 * nu/Re

        blf['u']['stokes'] = ...
        blf['p']['stokes'] = ...

        if not self.root.bcs.has_condition(Outflow):
            blf['p']['stokes'] += -mu * 1e-10 * U.p * V.p * ngs.dx

    def add_symbolic_convection_form(self, blf: Integrals, lf: Integrals):
        u, v = self.TnT['u']
        p, q = self.TnT['p']
        uhat, vhat = self.TnT['uhat']

        U = self.get_incompressible_state([u, p])
        V = self.get_incompressible_state([v, q])

        n = self.mesh.normal

        Uup = ngs.IfPos(U.u * n, U.u, (U.u*n) * n + self.tang(2*uhat-U.u))

        blf['u']['convection'] = ...
        blf['uhat']['convection'] = ...

    def set_boundary_conditions(self) -> None:
        inflows = {dom: bc.velocity for dom, bc in self.root.bcs.to_pattern(Inflow).items()}

        if inflows:
            u = self.mesh.BoundaryCF(inflows)
            self.gfus['u'].Set(u, ngs.BND)
            self.gfus['uhat'].Set(self.tang(u), ngs.BND)
```


you can copy most of the code from the Taylor-Hood finite element and make the corresponding changes.

### The spaces

We need the following spaces:

- For the velocity use the `ngs.HDiv` space of order `self.order` and Dirichlet boundary conditions on `bnds`
- For the (tangential) traces we use the `ngs.TangentialFacetFESpace` of order `self.order` and Dirichlet boundary conditions on `bnds`
- For the pressure we use the `ngs.L2` space of order `self.order-1`